In [6]:
import os

import random

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [7]:
# On déclare les chemins vers les données :

Image = 'Data'
        
test_data_dir = 'Data/TEST'
train_data_dir = 'Data/TRAIN'
validation_data_dir = 'Data/VALIDATION'

# Dimensions et path :
img_width, img_height = 224, 224

ImageTRAINCOVID = os.listdir(Image + '/TRAIN/A_COVID')
ImageTRAINAUTRE = os.listdir(Image + '/TRAIN/B_AUTRE')

ImageVALIDATIONCOVID = os.listdir(Image + '/VALIDATION/A_COVID')
ImageVALIDATIONAUTRE = os.listdir(Image + '/VALIDATION/B_AUTRE')

ImageTESTCOVID = os.listdir(Image + '/TEST/A_COVID')
ImageTESTAUTRE = os.listdir(Image + '/TEST/B_AUTRE')

print('Il y a ' + str(len(ImageTRAINCOVID)) + ' images d\'entrainement de patient covid.') 
print('Il y a ' + str(len(ImageTRAINAUTRE)) + ' images d\'entrainement de patient non-covid.') 
print('Il y a ' + str(len(ImageVALIDATIONCOVID)) + ' images de validation de patient covid.') 
print('Il y a ' + str(len(ImageVALIDATIONAUTRE)) + ' images de validation de patient non-covid.') 
print('Il y a ' + str(len(ImageTESTCOVID)) + ' images test de patient covid.') 
print('Il y a ' + str(len(ImageTESTAUTRE)) + ' images test de patient non-covid.') 

Il y a 800 images d'entrainement de patient covid.
Il y a 1200 images d'entrainement de patient non-covid.
Il y a 201 images de validation de patient covid.
Il y a 301 images de validation de patient non-covid.
Il y a 1143 images test de patient covid.
Il y a 2686 images test de patient non-covid.


In [8]:
# pré-processing

# On rescale les images :

datagen = ImageDataGenerator(rescale=1./255)

# On définit la batch size :

batch_size = 32 

# On prépare les tableaux de données depuis les images :

train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 502 images belonging to 2 classes.
Found 3829 images belonging to 2 classes.


In [9]:
# On définit l'architecture du modèle :

model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# On compile le modèle :

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# On affiche le modèle :

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_5 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_6 (Activation)    (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 52, 52, 64)       

In [10]:
# On définit les paramètres pour l'entrainement :
epochs = 100
train_samples = 2000
validation_samples = 502
test_samples = 3829

# On définit les callbacks : 
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min',patience = 10,restore_best_weights=True,),
]

In [11]:
# Entrainement du modèle :

historique = model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=epochs,
        callbacks = my_callbacks,
        validation_data=validation_generator,
        validation_steps=validation_samples// batch_size)

Epoch 1/100
62/62 [==============================] - 414s 7s/step - loss: 0.6456 - accuracy: 0.7978 - val_loss: 0.2973 - val_accuracy: 0.9083
Epoch 2/100
62/62 [==============================] - 330s 5s/step - loss: 0.1581 - accuracy: 0.9558 - val_loss: 0.3896 - val_accuracy: 0.8604
Epoch 3/100
62/62 [==============================] - 491s 8s/step - loss: 0.2352 - accuracy: 0.9522 - val_loss: 0.2499 - val_accuracy: 0.9167
Epoch 4/100
62/62 [==============================] - 616s 10s/step - loss: 0.0901 - accuracy: 0.9726 - val_loss: 0.2317 - val_accuracy: 0.9229
Epoch 5/100
62/62 [==============================] - 602s 10s/step - loss: 0.1078 - accuracy: 0.9721 - val_loss: 0.2590 - val_accuracy: 0.9167
Epoch 6/100
62/62 [==============================] - 303s 5s/step - loss: 0.0614 - accuracy: 0.9812 - val_loss: 0.1084 - val_accuracy: 0.9646
Epoch 7/100
62/62 [==============================] - 263s 4s/step - loss: 0.0715 - accuracy: 0.9817 - val_loss: 0.2858 - val_accuracy: 0.9167
Epoc

In [13]:
n_epochs = len(historique.history['loss'])
GenOptimale = n_epochs-10
print("la generation optimale est : ",GenOptimale)
model.save_weights('models/Weights/scratch_224_224_'+str(GenOptimale)+'.h5')

la generation optimale est :  6


In [14]:
def testImage(path):
    img = load_img(path, target_size=(224,224))
    img = img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    y = model.predict(img)
    return y[0][0]

In [15]:
# On évalue le modèle sur les données de validation :

resultValidation = model.evaluate_generator(validation_generator, validation_samples)

# On évalue le modèle sur les données de test :

resultTest = model.evaluate_generator(test_generator, test_samples)1

In [17]:
# On enregistre l'historique et les poids :

model.save_weights('models/scratch_10G_224_224.h5')

np.save('models/historique_scratch_224_224_10G_Baudouin.npy',historique.history)

model.save('models/scratchFull_10G_224_224')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/scratchFull_10G_224_224/assets
